# Optimized to (27 secs) SAP to SQL

In [ ]:
# sap_to_mysql_sync.py
# This script fetches data from an SAP OData service and stores it into a MySQL database.
# It first truncates the target table and then inserts all records from SAP.

import requests
from requests.auth import HTTPBasicAuth # Correct import for requests library
import mysql.connector
import json
import time
from datetime import datetime
import math

# --- Configuration ---
SAP_ODATA_URL = 'https://my412439-api.s4hana.cloud.sap/sap/opu/odata/sap/ZMM_60_BIN/ZMM_60_CDS'
SAP_USERNAME = 'ZNOVELSH412439'
SAP_PASSWORD = 'PEWMsSS$Pv3TSonFJlFUYJiEmcCfVmXcpzaMfeHw' # Ensure $ is properly handled if needed, or use raw string r"..."

DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASSWORD = ''
DB_NAME = 'simplexinternal'
DB_TABLE_NAME = 'item_groups'

PAGE_SIZE = 5000  # number of records per SAP page
BATCH_SIZE = 500  # number of records per MySQL insert batch

# Define the SAP fields you want to select from the API
# These names MUST match the field names in your SAP OData service
SAP_SELECT_FIELDS = [
    "Product",
    "YY1_MaterialLongDescri_PRD", # This will map to material_description
    "Plant",
    "ProductGroup",
    "ProductGroupName",
    "ExternalProductGroup",
    "ExternalProductGroupName"
    # Add other SAP fields here if you added more columns to your DB table AND want to populate them from SAP
    # For example, if your SAP service has "MaterialType" and "BaseUnit":
    # "MaterialType",
    # "BaseUnit"
]


# --- Helper functions ---

def log(msg):
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}")

def connect_to_mysql():
    return mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        autocommit=False, # Batch inserts benefit from manual commit
    )

def clean_record_value(value):
    # Convert value to string if it's not None, otherwise keep it None
    if value is None:
        return None
    if isinstance(value, dict): # OData can have nested structures like __deferred or results for complex types
        # Decide how to handle dictionaries. For now, skip them or take a specific sub-field if known.
        # If it's a complex type you don't want to store directly, return None or a placeholder.
        # Example: return json.dumps(value) # if you want to store as JSON string
        return None # Or skip if these are not meant for direct column storage
    return str(value)


def get_total_record_count():
    # Use a session for consistency
    with requests.Session() as session:
        session.auth = HTTPBasicAuth(SAP_USERNAME, SAP_PASSWORD)
        session.headers.update({'Accept': 'application/json'})
        
        url = f"{SAP_ODATA_URL}/$count" # OData standard for getting count ($inlinecount is for results + count)
                                        # Some V2 services might need entity set + /$count or $inlinecount with $top=0
        
        # Fallback to $inlinecount if /$count is not supported
        inline_count_url = f"{SAP_ODATA_URL}?$top=0&$inlinecount=allpages&$format=json"

        log(f"Fetching total record count from SAP (attempt 1: /$count)... {url}")
        response = None
        try:
            response = session.get(url, timeout=60)
            log(f"HTTP Status (Total Count Request via /$count): {response.status_code}")
            if response.status_code == 200:
                try:
                    # For /$count, the response is often just the plain number
                    count = int(response.text)
                    log(f"Total records available (via /$count): {count}")
                    return count
                except ValueError:
                    log("  /$count did not return a plain number. Trying JSON response...")
                    data = response.json() # Try parsing as JSON if direct int conversion fails
                    if '@odata.count' in data: # OData V4 count
                        count = int(data['@odata.count'])
                        log(f"Total records available (via @odata.count): {count}")
                        return count
                    elif 'd' in data and '__count' in data['d']: # OData V2 count with $inlinecount trick
                         count = int(data['d']['__count'])
                         log(f"Total records available (via d.__count with /$count URL): {count}")
                         return count


        except requests.exceptions.RequestException as e:
            log(f"  Failed /$count: {e}. Trying $inlinecount method.")
        except json.JSONDecodeError as e:
             log(f"  Failed to decode JSON from /$count response: {e}. Trying $inlinecount method.")


        log(f"Fetching total record count from SAP (attempt 2: $inlinecount)... {inline_count_url}")
        response = session.get(inline_count_url, timeout=60)
        log(f"HTTP Status (Total Count Request via $inlinecount): {response.status_code}")
        if response.status_code != 200:
            raise Exception(f"Failed to get total count from SAP. Status code: {response.status_code}")

        data = response.json()
        if 'd' in data and '__count' in data['d']:
            count = int(data['d']['__count'])
            log(f"Total records available (via $inlinecount): {count}")
            return count
        else:
            log(f"Could not determine total count using $inlinecount. Response: {str(data)[:200]}")
            raise Exception("Failed to get total count using $inlinecount from SAP.")


def fetch_data_from_sap(total_count):
    records = []
    # Calculate number of pages based on total_count and PAGE_SIZE
    pages = (total_count + PAGE_SIZE - 1) // PAGE_SIZE if total_count > 0 else 0
    if total_count == 0: # If total_count is 0, no need to fetch
        log("Total record count is 0. No data to fetch.")
        return records
        
    log(f"Starting full data fetch, total pages to fetch: {pages}...")

    current_url = SAP_ODATA_URL # Base URL for the entity set

    # Construct initial query parameters
    query_params = {
        '$format': 'json',
        '$top': PAGE_SIZE
    }
    if SAP_SELECT_FIELDS: # Add $select if fields are specified
        query_params['$select'] = ",".join(SAP_SELECT_FIELDS)

    # Use a session for all data requests
    with requests.Session() as session:
        session.auth = HTTPBasicAuth(SAP_USERNAME, SAP_PASSWORD)
        session.headers.update({'Accept': 'application/json'})

        for page in range(pages):
            skip = page * PAGE_SIZE
            query_params['$skip'] = skip # Add/update $skip for current page

            # Build URL with parameters for the current page
            current_page_url_with_params = current_url + "?" + "&".join([f"{k}={v}" for k,v in query_params.items()])
            
            log(f"Fetching page {page+1}/{pages} from SAP: {current_page_url_with_params.split('?')[0]}...")
            try:
                response = session.get(current_page_url_with_params, timeout=300)
                log(f"  HTTP Status: {response.status_code}")
                response.raise_for_status()

                page_data = response.json()
                
                # The structure of OData V2 responses usually has results under 'd.results'
                page_records = page_data.get('d', {}).get('results', [])
                
                log(f"  Retrieved {len(page_records)} records from page {page+1}")
                records.extend(page_records)

                # OData V2 uses 'd.__next' for the next page link.
                # If we are manually calculating pages with $skip, we don't strictly need __next here,
                # but it's a good check. The loop is based on calculated pages.
                next_link = page_data.get('d', {}).get('__next')
                if not next_link and (page + 1) < pages:
                    log(f"  Warning: No __next link found on page {page+1}, but more pages were expected.")

            except requests.exceptions.RequestException as e:
                log(f"  Failed to fetch data from SAP page {page+1}, Error: {e}")
                if hasattr(e, 'response') and e.response is not None:
                    log(f"  Response status: {e.response.status_code}")
                    log(f"  Response text: {e.response.text[:500]}")
                break # Stop fetching on error
            except json.JSONDecodeError as e:
                log(f"  Failed to decode JSON from SAP page {page+1}, Error: {e}")
                log(f"  Response text: {response.text[:500] if 'response' in locals() else 'Response object not available'}")
                break

    log(f"Fetched total records: {len(records)}")
    return records


def insert_data_to_mysql(data):
    if not data:
        log("No data to insert.")
        return

    conn = connect_to_mysql()
    cursor = conn.cursor()

    log(f"Connected to MySQL. Truncating table '{DB_TABLE_NAME}'...")
    cursor.execute(f"TRUNCATE TABLE `{DB_TABLE_NAME}`")
    log("Table truncated.")

    # --- Define the mapping from SAP field names to your DB column names ---
    # This also defines the order of columns in your INSERT statement.
    # DB Column Name              SAP Field Name in 'record'
    # Ensure these DB columns exist in your item_groups table
    column_mapping = {
        "material_no":              "Product",
        "material_description":     "YY1_MaterialLongDescri_PRD",
        "plant":                    "Plant",
        "material_group_no":        "ProductGroup",
        "material_group_name":      "ProductGroupName",
        "external_group_no":        "ExternalProductGroup",
        "external_group_name":      "ExternalProductGroupName"
        # "type":                     "MaterialType", # Example if you add this back
        # "uom":                      "BaseUnit"      # Example if you add this back
    }
    
    db_columns = list(column_mapping.keys())
    sap_source_fields = list(column_mapping.values())

    col_names_sql = ', '.join([f"`{col}`" for col in db_columns])
    placeholders = ', '.join(['%s'] * len(db_columns))
    insert_query = f"INSERT INTO `{DB_TABLE_NAME}` ({col_names_sql}) VALUES ({placeholders})"

    log(f"Preparing to insert {len(data)} records in batches of {BATCH_SIZE}...")
    
    insert_count = 0
    error_count = 0
    
    try:
        for i in range(0, len(data), BATCH_SIZE):
            batch_records = data[i:i + BATCH_SIZE]
            values_to_insert = []
            for record in batch_records:
                # Clean each SAP record and prepare tuple in the order of db_columns
                cleaned_item = {}
                for sap_field in SAP_SELECT_FIELDS: # Iterate only through fields we actually selected
                    cleaned_item[sap_field] = clean_record_value(record.get(sap_field))

                # Build the tuple based on the order of db_columns and mapping
                current_tuple = []
                for db_col in db_columns:
                    sap_field_for_this_db_col = column_mapping.get(db_col)
                    current_tuple.append(cleaned_item.get(sap_field_for_this_db_col))
                values_to_insert.append(tuple(current_tuple))
            
            if values_to_insert:
                cursor.executemany(insert_query, values_to_insert)
                conn.commit()
                insert_count += len(values_to_insert)
                log(f"  Inserted records {i + 1} to {i + len(values_to_insert)}. Total inserted: {insert_count}")
            else:
                log(f"  Batch {i // BATCH_SIZE + 1} was empty after cleaning/mapping.")

    except mysql.connector.Error as err:
        log(f"MySQL error during batch insert: {err}")
        conn.rollback() # Rollback on error
        error_count = len(data) - insert_count # Estimate errors
    except Exception as e:
        log(f"General error during batch insert: {e}")
        conn.rollback()
        error_count = len(data) - insert_count # Estimate errors
    finally:
        cursor.close()
        conn.close()
        log(f"MySQL connection closed. Total Inserted: {insert_count}, Estimated Errors: {error_count}")

# --- Main script ---

def main():
    log("--- SAP to MySQL Sync Script Started ---")
    start_time = time.time()

    try:
        total_count = get_total_record_count()
        if total_count > 0:
            data = fetch_data_from_sap(total_count)
            insert_data_to_mysql(data)
        else:
            log("SAP service reported 0 records. No data to sync.")
            # If count is 0, still truncate the table if that's desired behavior
            conn = connect_to_mysql()
            cursor = conn.cursor()
            log(f"Connected to MySQL. Truncating table '{DB_TABLE_NAME}' as total count from SAP is 0...")
            cursor.execute(f"TRUNCATE TABLE `{DB_TABLE_NAME}`")
            conn.commit()
            log("Table truncated.")
            cursor.close()
            conn.close()

    except Exception as e:
        log(f"An error occurred in the main process: {e}")

    elapsed = time.time() - start_time
    log(f"--- Sync Script Finished ---")
    log(f"--- Total Execution Time: {elapsed:.2f} seconds ---")

if __name__ == "__main__":
    main()

